### Ноутбук с обучением и тестированием модели
- Тестирование в конце

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Библиотеки

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [0]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [0]:
corpus_name = "pikabu"
corpus = "pikabu"
datafile = "pikabu.csv"

In [0]:
import pandas as pd

### Предобработка

In [0]:
data = pd.read_csv('pikabu.csv')

In [0]:
data.head()

In [0]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

In [0]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

        for word in keep_words:
            self.addWord(word)

In [0]:
MAX_LENGTH = 15

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [0]:
def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^а-яА-Я.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    s = s.lower()
    return s

In [0]:
from tqdm import tqdm

In [0]:
tqdm.pandas(desc="my bar!")

In [15]:
import swifter

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
data['context'] = data['context'].swifter.apply(normalizeString)

In [0]:
data['answer'] = data['answer'].swifter.apply(str)

In [0]:
data['answer'] = data['answer'].swifter.apply(normalizeString)

In [0]:
data.head()

In [0]:
data.to_csv('pikabu.csv', index=False)

In [0]:
data = pd.read_csv('gdrive/My Drive/Colab Notebooks/pikabu.csv')

In [85]:
data.head()

,context,answer
0,убрал литров мусора в реликтовом лесу . ты про...,спасибо .
1,убрал литров мусора в реликтовом лесу . ты про...,приедь к нам в мурманск пожалуйста .
2,убрал литров мусора в реликтовом лесу . ты про...,тебе платят за это ?
3,ты просто большой молодец . спасибо . тебе пла...,за посты на пикабу ? да платят . достаточно пр...
4,спасибо . тебе платят за это ? за посты на пик...,не нажимается сук


In [0]:
def readVocs(data, corpus_name):
    print("Reading lines...")
    
    pairs = []
    
    for row in range(len(data)):
        pairs.append([data.iloc[row]['context'],
                      data.iloc[row]['answer']])
        if row % 10000 == 0:
            print(row)
    voc = Voc(corpus_name)
    
    return voc, pairs

In [0]:
def filterPair(p):
    try:
        return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH
    except AttributeError:
        return False

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def loadPrepareData(corpus, corpus_name, data, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(data, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

In [0]:
data = data.sample(600000)

In [0]:
import tqdm

In [28]:
voc, pairs = readVocs(data, corpus_name)

Reading lines...
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000


In [0]:
pairs = filterPairs(pairs)

In [0]:
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

In [31]:
print("Counted words:", voc.num_words)

Counted words: 140846


In [33]:
len(pairs)

131940

In [0]:
import pickle

In [0]:
# with open('pairs.pickle', 'wb') as handle:
#     pickle.dump(pairs, handle)

In [0]:
# with open('pairs.pickle', 'rb') as handle:
#     pairs = pickle.load(handle)

In [0]:
# with open('voc.pickle', 'wb') as handle:
#     pickle.dump(voc, handle)

In [0]:
# with open('voc.pickle', 'rb') as handle:
#     voc = pickle.load(handle)

In [0]:
MIN_COUNT = 2    #Встречаемость слов

def trimRareWords(voc, pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

In [41]:
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 65370 / 140843 = 0.4641
Trimmed from 131940 pairs to 82206, 0.6231 of total


In [0]:
import pickle

In [0]:
with open('gdrive/My Drive/Colab Notebooks/pairs.pickle', 'wb') as handle:
    pickle.dump(pairs, handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/pairs.pickle', 'rb') as handle:
    pairs = pickle.load(handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/voc.pickle', 'wb') as handle:
    pickle.dump(voc, handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/voc.pickle', 'rb') as handle:
    voc = pickle.load(handle)

In [47]:
len(pairs)

82206

In [0]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [51]:
#Пример батча
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[16866, 12627,   514,  1300, 36552],
        [ 7636,  2915,    81,   265,     2],
        [   98,    10, 20539,   703,     0],
        [ 1166,  1183,    24,     2,     0],
        [   21, 27241,  5605,     0,     0],
        [  480,    24,    95,     0,     0],
        [   63, 24164,    22,     0,     0],
        [   23,   844,   165,     0,     0],
        [   40,  3798,     2,     0,     0],
        [ 5511,    24,     0,     0,     0],
        [  105,     2,     0,     0,     0],
        [ 1806,     0,     0,     0,     0],
        [15583,     0,     0,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([14, 11,  9,  4,  2])
target_variable: tensor([[  374,   105,    99, 10333,   824],
        [   22,    96,   645,    24,    10],
        [   23, 19473, 15249,   172,  9701],
        [15583,     2,    81,    24,  4554],
        [    2,     0,  1993,     2,  3295],
        [    0,     0, 20148,     0,    24],
        [    0,     0,     2

### Модели

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

In [0]:
# Luong attention
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()

        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [0]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.gru(embedded, last_hidden)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

In [0]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [0]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    loss.backward()

    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [0]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [0]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        return all_tokens, all_scores

In [0]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # Слова -> индексы
    indexes_batch = [indexesFromSentence(voc, sentence)]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Декодируем предложения
    tokens, scores = searcher(input_batch, lengths, max_length)
    # индексы -> слова
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            input_sentence = input('> ')
            # Выход из чата
            if input_sentence == 'q' or input_sentence == 'quit': break
            input_sentence = normalizeString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [0]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

In [0]:
loadFilename = None
checkpoint_iter = 4000

In [65]:

if loadFilename:
    checkpoint = torch.load(loadFilename)
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [0]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

In [0]:
save_dir = os.path.join("data", "save")

In [70]:
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 11.0975
Iteration: 2; Percent complete: 0.1%; Average loss: 11.0484
Iteration: 3; Percent complete: 0.1%; Average loss: 10.9950
Iteration: 4; Percent complete: 0.1%; Average loss: 10.9040
Iteration: 5; Percent complete: 0.1%; Average loss: 10.7720
Iteration: 6; Percent complete: 0.1%; Average loss: 10.5204
Iteration: 7; Percent complete: 0.2%; Average loss: 10.0854
Iteration: 8; Percent complete: 0.2%; Average loss: 9.6525
Iteration: 9; Percent complete: 0.2%; Average loss: 9.0898
Iteration: 10; Percent complete: 0.2%; Average loss: 9.0588
Iteration: 11; Percent complete: 0.3%; Average loss: 9.3011
Iteration: 12; Percent complete: 0.3%; Average loss: 9.1711
Iteration: 13; Percent complete: 0.3%; Average loss: 9.1255
Iteration: 14; Percent complete: 0.4%; Average loss: 8.8701
Iteration: 15; Percent complete: 0.4%; Average loss: 8.4295
Iteration: 16; Percent complet

In [0]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

### Запуск

In [0]:
evaluateInput(encoder, decoder, searcher, voc)

### Для telegram пока будем хранить voc, encoder, decoder
- В нем запускаем evaluateInput(encoder, decoder, searcher, voc) и все необходимые в ней

In [0]:
import pickle

In [0]:
with open('gdrive/My Drive/Colab Notebooks/encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/decoder.pickle', 'wb') as handle:
    pickle.dump(decoder, handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/voc.pickle', 'wb') as handle:
    pickle.dump(voc, handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/voc.pickle', 'rb') as handle:
    voc = pickle.load(handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [0]:
with open('gdrive/My Drive/Colab Notebooks/decoder.pickle', 'rb') as handle:
    decoder = pickle.load(handle)

In [0]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

In [0]:
evaluateInput(encoder, decoder, searcher, voc)

In [82]:
evaluateInput(encoder, decoder, searcher, voc)

> Привет
Bot: привет
> Как ты?
Bot: я не знаю что ты как ?
> Я хорошо
Bot: я тебя люблю тебя
> Спасибо!
Bot: спасибо
> Я тоже тебя люблю
Bot: я тебя люблю тебя
> Спокойной ночи
Bot: спокойной ночи
> Сладких снов
Bot: доброй ночи
> Я хочу спать
Bot: я не хочу
> Почему?
Bot: потому что я не знаю что это не знаю
> Ты из Москвы?
Bot: я не знаю что ты не видела
> Москва
Bot: в смысле ?
> В прямом
Bot: а что за фильм ?
> Российский
Bot: а я не знаю что это не так
> Какая музыка хорошая?
Bot: а ты не спишь ?
> Не сплю
Bot: не помню
> нормас
Bot: а что за мем ? . . .
> ...
Bot: . . . . . . . . не очень ! !
> Удачи тебе
Bot: спасибо
> Ты там держись удачи и хорошего настроения
Bot: спасибо
> q
